# Query Cell Types Ontology

# Context

- This notebook has been put together for the MMB demo on 2022-05-30: see [slides](https://docs.google.com/presentation/d/1Ib1_8byK0hVuNS-wPbqmeL5Lcf67m_oegC3-czPw5ws/edit#slide=id.g116ba5ed71e_0_8)
- It has been revised following feedback from the meeting held on 2022-07-07: see [slides](https://docs.google.com/presentation/d/1mgCyYjHerLJLV79GM0kqp3_Htxmru_7QGIr5elUETC0/edit#slide=id.g13b4a370a10_0_19) and [JIRA ticket](https://bbpteam.epfl.ch/project/issues/browse/DKE-942)

## Imports

In [ ]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup

In [ ]:
TOKEN = getpass.getpass()

The cell type ontology is stored in the `neurosciencegraph/datamodels` bucket in the knowledge graph. This is the `bucket` we target with the forge instance below.

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="neurosciencegraph/datamodels")

## Ontologies

### Set brain region

During the meeting on `2022-05-30`, it was specified that a brain region will serve as entry point when searching for cell types in the MMB context. Hence, this notebook starts by defining a brain region one wants to get cell types for. Since the most complete cell type information is available for the `Cerebral cortex`, this has been set as the default below.

In [ ]:
BRAIN_REGION = "Cerebral cortex"
# BRAIN_REGION = "Cerebellum"

Get brain region id

In [ ]:
r = forge.search({"label": BRAIN_REGION})

In [ ]:
brain_region = r[0].id

In [ ]:
brain_region

## Queries

### Get brain regions which do have neuron t-types available

This query will list brain region labels for which the knowledge graph has neuron t-types available

In [ ]:
query = f"""

SELECT ?brain_region

WHERE {{
        ?t_type_id subClassOf* <https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType> ;
                  canHaveBrainRegion ?brain_region_id .        
        ?brain_region_id label ?brain_region .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=1000)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df

In [ ]:
set(df.brain_region)

### Get possible t-types for a given brain region

This query lists t-types for a given brain region (i.e. the `BRAIN_REGION` specified above). For demonstration purposes, the `limit` parameter on the query has been set to `100`. This can be increased to get all available t-types. E.g. the total number of available t-types for `Cerebral cortex` on `2022-07-08` was `252`.

In [ ]:
query = f"""

SELECT ?brain_region ?t_type

WHERE {{
        ?t_type_id label ?t_type ;
            subClassOf* <https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType> ;
            canHaveBrainRegion <{brain_region}> .        
        <{brain_region}> label ?brain_region . 
}}
"""

In [ ]:
resources = forge.sparql(query, limit=1000)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get possible met-type combinations plus excitatory/inhibitory category for a given brain region

This query returns possible met-type combinations together with the excitatory/inhibitory categories for the brain region one has set above. For each m- e and t- and transmitter-type, the identifier and the current version in the knowledge graph are also being returned. For a simplified view, please run the `df.drop()` cell below. It will only show the labels of a given type. The `version` indicates the revision of a given type in the knowledge graph and has been included following the Cell Types Meeting on 2022-07-07 to help with reproducibility (see also this JIRA ticket: [DKE-942](https://bbpteam.epfl.ch/project/issues/browse/DKE-942)).

In [ ]:
query = f"""

SELECT ?brain_region ?brain_region_version ?transmitter ?transmitter_id ?transmitter_version ?t_type ?t_type_id ?t_type_version ?m_type ?m_type_id ?m_type_version ?e_type ?e_type_id ?e_type_version

WHERE {{
        ?t_type_id label ?t_type ;
                canHaveBrainRegion <{brain_region}> ;
                _rev ?t_type_version .
        
        <{brain_region}> label ?brain_region ;
            _rev ?brain_region_version .
        
        ?m_type_id label ?m_type ;
            _rev ?m_type_version ;
            subClassOf* MType ;
            canHaveTType ?t_type_id ;
            subClassOf* / hasNeurotransmitterType ?transmitter_id .
        
        ?transmitter_id label ?transmitter ;        
            _rev ?transmitter_version .

        ?e_type_id label ?e_type ;
            _rev ?e_type_version ;
            subClassOf* EType ;
            subClassOf* / canHaveMType ?m_type_id ;
            subClassOf* / canHaveTType ?t_type_id .            
}}
"""

In [ ]:
resources = forge.sparql(query, limit=1000)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

In [ ]:
df.drop(["brain_region_version", "e_type_id", "e_type_version", "m_type_id", "m_type_version", "t_type_id", "t_type_version", "transmitter_id", "transmitter_version"], axis=1)

### Get possible t-types for a given brain region and all the brain regions which are part of that brain region

This query returns possible t-types for the brain region one has set above and all the brain regions that are part of that brain region. E.g. if one specifies `Cerebral cortex` as brain region, this query would return t-types from the `Cerebral cortex` but also t-types for the `Isocortex` or the `Hippocampal formation` since they are both part of the `Cerebral cortex`.

In [ ]:
query = f"""

SELECT ?brain_region ?brain_region_version ?t_type ?t_type_id ?t_type_version

WHERE {{
        ?t_type_id label ?t_type ;
                canHaveBrainRegion ?brain_region_id ;
                _rev ?t_type_version .
        
        ?brain_region_id label ?brain_region ;
            ^hasPart* <{brain_region}> ;
            _rev ?brain_region_version .            
}}
"""

In [ ]:
resources = forge.sparql(query, limit=500)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
len(set(df.t_type))

In [ ]:
df

### Get m-types together with their transmitter type

In [ ]:
query = f"""

SELECT ?transmitter ?m_type

WHERE {{
        ?m_type_id label ?m_type ;
            subClassOf* MType ;
            subClassOf* / hasNeurotransmitterType / label ?transmitter .           
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get m-types with a given morphology and the morphology definition

This query returns m-types which have a given morphological shape. The cell morphologies were taken from the [Phenotype and Trait Ontology](https://ontobee.org/ontology/PATO) (this was done following the request of Georges Khazen who wanted to include the `PATO` deinfitions of morphologies).
Set `MORPHOLOGY` below to one of the following:

- `standard pyramidal morphology`
- `pyramidal family morphology`
- `tufted pyramidal morphology`
- `basket cell morphology`
- `chandelier cell morphology`
- `neurogliaform morphology`
- `Martinotti morphology`
- `cortical bipolar morphology`
- `bitufted cell morphology`

In [ ]:
MORPHOLOGY = "basket cell morphology"

In [ ]:
query = f"""

SELECT ?cell ?definition

WHERE {{
        ?cell_id subClassOf* / hasMorphologicalPhenotype ?pato_id ;
                  label ?cell .
        ?pato_id subClassOf* / label "{MORPHOLOGY}" .
        ?parent_pato_id label "{MORPHOLOGY}" ;
                <http://purl.obolibrary.org/obo/IAO_0000115> ?definition .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

In [ ]:
df

### Get t-types from a paper specific paper

The [Cell Types and Missing Data - Version 1](https://docs.google.com/spreadsheets/d/1iUgqPszKkYQgkJlmpQSkeyFWcEoOxovsBkoLPtA3qPg/edit#gid=1180597294) spreadsheet which served as source for the `Cell Types Ontology` - lists paper identifiers on the `Notes` sheet. These references were added to the respective t-types.
Set `PAPER` below to one of the following:

- Yao et al. 2021: `https://www.sciencedirect.com/science/article/pii/S0092867421005018?dgcid=rss_sd_all`
- Gokce 2016: `https://www.ncbi.nlm.nih.gov/labs/pmc/articles/PMC5004635/`
- Kozareva et al. 2021: `https://www.nature.com/articles/s41586-021-03220-z`
- Chen et al. 2017: `https://www.sciencedirect.com/science/article/pii/S2211124717303212?via%3Dihub`
- Kalish et al. 2018: `https://www.pnas.org/content/115/5/E1051`

In [ ]:
PAPER = "https://www.sciencedirect.com/science/article/pii/S0092867421005018?dgcid=rss_sd_all"

In [ ]:
query = f"""

SELECT ?label ?brain_region

WHERE {{
        ?id seeAlso <{PAPER}> ;
            label ?label .
        OPTIONAL {{ ?id canHaveBrainRegion / label ?brain_region }} .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get the m- e- and t-type placeholders

One of the requirements specified during the meeting on `2022-05-30` was to have a placeholder class for each of the types. We thus implemented an m- e- and t-type placeholder class.

In [ ]:
query = """

SELECT ?id ?label

WHERE {
        ?id label ?label .
        FILTER (CONTAINS(STR(?label), 'Placeholder'))
}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df

### Get all cell type combinations and probabilities for a given brain region

This query cell type combinations for a given brain region (i.e. the `BRAIN_REGION` specified above). For demonstration purposes, the `limit` parameter on the query has been set to `1000`. This can be increased to get all available cell type combinations.

In [ ]:
query = f"""

SELECT ?brain_region ?m_type ?e_type ?molecular_type ?probability

WHERE {{
        ?probability_id hasTarget / hasSource / hasSomaLocatedIn ?brain_region_id ;
            hasBody / value ?probability ;
            hasTarget ?m_type_target ;
            hasTarget ?e_type_target ;
            hasTarget ?molecular_type_target .
        ?brain_region_id label ?brain_region ;
            ^hasPart* <{brain_region}> .
        ?m_type_target hasSource / a MType ;
            hasSource / label ?m_type .
        ?e_type_target hasSource / a EType ;
            hasSource / label ?e_type .
        ?molecular_type_target hasSource / a NeuronMolecularType ;
            hasSource / label ?molecular_type .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=1000)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()